In [2]:
# Sample code from https://www.redblobgames.com/pathfinding/a-star/
# Copyright 2014 Red Blob Games <redblobgames@gmail.com>
#
# Feel free to use this code in your own projects, including commercial projects
# License: Apache v2.0 <http://www.apache.org/licenses/LICENSE-2.0.html>

from __future__ import annotations
# some of these types are deprecated: https://www.python.org/dev/peps/pep-0585/
from typing import Protocol, Iterator, Tuple, TypeVar, Optional
T = TypeVar('T')

Location = TypeVar('Location')
class Graph(Protocol):
    def neighbors(self, id: Location) -> list[Location]: pass

class SimpleGraph:
    def __init__(self):
        self.edges: dict[Location, list[Location]] = {}

    def neighbors(self, id: Location) -> list[Location]:
        return self.edges[id]

import collections

# utility functions for dealing with square grids
def from_id_width(id, width):
    return (id % width, id // width)

def draw_tile(graph, id, style):
    r = " ⋅ "
    if id in graph.swamps: r = "⋅⋅⋅"
    if 'number' in style and id in style['number']: r = " %-2d" % style['number'][id]
    if 'point_to' in style and style['point_to'].get(id, None) is not None:
        (x1, y1) = id
        (x2, y2) = style['point_to'][id]
        if x2 == x1 + 1: r = " > "
        if x2 == x1 - 1: r = " < "
        if y2 == y1 + 1: r = " ∨ "
        if y2 == y1 - 1: r = " ∧ "
    if 'path' in style and id in style['path']:   r = " @ "
    if 'start' in style and id == style['start']: r = " S "
    if 'goal' in style and id == style['goal']:   r = " G "
    if id in graph.walls: r = "###"
    return r

def draw_grid(graph, **style):
    print("___" * graph.width)
    for y in range(graph.height):
        for x in range(graph.width):
            print("%s" % draw_tile(graph, (x, y), style), end="")
        print()
    print("---" * graph.width)

# data from main article
DIAGRAM1_WALLS = [from_id_width(id, width=30) for id in [21,22,51,52,81,82,93,94,111,112,123,124,133,134,141,142,153,154,163,164,171,172,173,174,175,183,184,193,194,201,202,203,204,205,213,214,223,224,243,244,253,254,273,274,283,284,303,304,313,314,333,334,343,344,373,374,403,404,433,434]]
GridLocation = Tuple[int, int]

class SquareGrid:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.walls: list[GridLocation] = []
        self.swamps: list[GridLocation] = []

    def in_bounds(self, id: GridLocation) -> bool:
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height

    def passable(self, id: GridLocation) -> bool:
        return id not in self.walls

    def neighbors(self, id: GridLocation) -> Iterator[GridLocation]:
        (x, y) = id
        neighbors = [(x+1, y), (x-1, y), (x, y-1), (x, y+1)] # E W N S
        # see "Ugly paths" section for an explanation:
        if (x + y) % 2 == 0: neighbors.reverse() # S N W E
        results = filter(self.in_bounds, neighbors)
        results = filter(self.passable, results)
        return results

def reconstruct_path(came_from: dict[Location, Location],
                    start: Location, goal: Location) -> list[Location]:

    current: Location = goal
    path: list[Location] = []
    if goal not in came_from: # no path was found
        return []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start) # optional
    path.reverse() # optional
    return path

g = SquareGrid(30, 15)
g.walls = DIAGRAM1_WALLS

start = (8, 7)
goal = (18, 8)

draw_grid(g, start=start, goal=goal)

__________________________________________________________________________________________
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ############### ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ############### ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  S  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  G  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 

Uninformed Search

In [3]:
class FIFO:
    """A simple queue implementation using `collections.deque`."""
    def __init__(self) -> None:
        pass

def breadth_first_search(graph: Graph, start: Location, goal: Location):
    pass

In [4]:
#test bfs
parents = breadth_first_search(g, start, goal)
draw_grid(g, point_to=parents, start=start, goal=goal)
draw_grid(g, path=reconstruct_path(parents, start=start, goal=goal),start=start, goal=goal)

__________________________________________________________________________________________


AttributeError: 'NoneType' object has no attribute 'get'

Informed Search

In [ ]:
from os import get_blocking
class WeightedGraph(Graph):
    def cost(self, from_id: Location, to_id: Location) -> float: pass

class GridWithWeights(SquareGrid):
    def __init__(self, width: int, height: int):
        super().__init__(width, height)
        self.weights: dict[GridLocation, float] = {}

    def cost(self, from_node: GridLocation, to_node: GridLocation) -> float:
        if to_node in self.swamps:
            return self.weights.get(to_node, 5)
        return self.weights.get(to_node, 1)


D4_walls = [(1, 7), (1, 8), (2, 7), (2, 8), (3, 7), (3, 8)]
D4_swamp = {loc: 5 for loc in [(3, 4), (3, 5), (4, 1), (4, 2),
                                       (4, 3), (4, 4), (4, 5), (4, 6),
                                       (4, 7), (4, 8), (5, 1), (5, 2),
                                       (5, 3), (5, 4), (5, 5), (5, 6),
                                       (5, 7), (5, 8), (6, 2), (6, 3),
                                       (6, 4), (6, 5), (6, 6), (6, 7),
                                       (7, 3), (7, 4), (7, 5)]}


start, goal = (1, 4), (8, 3)
g = GridWithWeights(11, 10)
g.walls = D4_walls
g.swamps = D4_swamp
draw_grid(g, start=start, goal=goal)

_________________________________
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ G  ⋅  ⋅ 
 ⋅  S  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅ #########⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅ #########⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
---------------------------------
